
# AI-Powered Personalized Weight Plan (with PDF Export)

This notebook collects basic user inputs to create a customized gym, calorie, and step-count schedule.
It outputs a 20–30 day plan and saves the result as a **PDF file named after the user**.


In [ ]:

import pandas as pd
import random
from itertools import cycle
from fpdf import FPDF


In [ ]:

# Collect inputs
username = input("Enter your name: ").strip()
current_weight = float(input("Enter your current weight (kg): "))
target_weight = float(input("Enter your target weight (kg): "))
num_days = int(input("Enter number of days to achieve your goal: "))


In [ ]:

# Read workout data
df_workouts = pd.read_csv("Workout_Dataset.csv")

# Determine goal
if target_weight < current_weight:
    goal = "reduce"
elif target_weight > current_weight:
    goal = "increase"
else:
    goal = "maintain"

# Calorie & step logic
if goal == "reduce":
    cal_min, cal_max = 1600, 1800
    step_range = (7500, 10000)
elif goal == "increase":
    cal_min, cal_max = 2200, 2500
    step_range = (5000, 8000)
else:
    cal_min, cal_max = 1800, 2200
    step_range = (7000, 9000)

# Group workouts
strength_df = df_workouts[df_workouts["Type"] == "Strength"]
cardio_df = df_workouts[df_workouts["Type"] == "Cardio"]
strength_dict = strength_df.groupby("Muscle")["Workout Name"].apply(list).to_dict()
muscle_groups = cycle(strength_dict.keys())

# Generate the plan
plan = []
for i in range(num_days):
    day = i + 1
    if day % 5 == 0:
        workout_name, muscle, sets_time = "Rest + Walking", "-", "-"
    else:
        if i % 2 == 0:
            m1 = next(muscle_groups)
            m2 = next(muscle_groups)
            w1 = random.choice(strength_dict[m1])
            w2 = random.choice(strength_dict[m2])
            workout_name = f"{w1} & {w2}"
            muscle = f"{m1} & {m2}"
            sets_time = "3-4 sets (12 reps) each"
        else:
            row = cardio_df.sample(1).iloc[0]
            workout_name = row["Workout Name"]
            muscle = row["Muscle"]
            sets_time = row["Reps/Time"]
    steps = random.randint(*step_range)
    calories = random.randint(cal_min, cal_max)
    plan.append([day, workout_name, muscle, sets_time, steps, f"{calories} kcal"])

# Create DataFrame
columns = ["Day", "Workout Name", "Target Muscle", "Sets / Time", "Steps Count", "Calories Intake"]
df = pd.DataFrame(plan, columns=columns)
df


In [ ]:

# Generate PDF
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.set_auto_page_break(auto=True, margin=15)

pdf.set_font("Arial", 'B', 14)
pdf.cell(200, 10, txt=f"{username}'s Weight Plan", ln=True, align='C')
pdf.ln(5)

pdf.set_font("Arial", 'B', 10)
headers = df.columns.tolist()
col_widths = [10, 50, 35, 35, 25, 30]
for i, header in enumerate(headers):
    pdf.cell(col_widths[i], 10, header, border=1)
pdf.ln()

pdf.set_font("Arial", size=9)
for _, row in df.iterrows():
    for i, item in enumerate(row):
        text = str(item).replace("—", "-")[:30]
        pdf.cell(col_widths[i], 10, text, border=1)
    pdf.ln()

# Save PDF
pdf_filename = f"{username.lower().strip()}.pdf"
pdf.output(pdf_filename)
print(f"✅ PDF saved as {pdf_filename}")
